In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import netCDF4 as nc

# !pip install xarray
import xarray as xr

# Introdução

[Referência para a implementação do trabalho](https://datascience-enthusiast.com/R/R_netcdf_correlation.html)


## Links para os datasets
* [Precipition - Mean - Surface - Monthly](https://psl.noaa.gov/data/gridded/data.gpcp.html)
* [Air Temperature - Mean - Surface - Monthly](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html)


##### Lendo os datasets

In [50]:
DATA_DIR = "./data"

precip = xr.open_dataset(f"{DATA_DIR}/precip.mon.mean.nc")
air = xr.open_dataset(f"{DATA_DIR}/air.sig995.mon.mean.nc")

precip

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 1332)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 1900-01-01 1900-02-01 ... 2010-12-01
Data variables:
    precip   (time, lat, lon) float32 ...
Attributes:
    history:        created 8/2012 by CAS NOAA/ESRL PSD
    title:          Univ. Delaware Monthly Precipitation, 1900-2010 V3.01
    Conventions:    CF-1.0
    Source:         http://climate.geog.udel.edu/~climate/html_pages/Global2_...
    version:        3.01
    dataset_title:  Terrestrial Air Temperature and Precipitation: 1900-2014 ...
    source:         http://climate.geog.udel.edu/~climate/
    References:     https://www.psl.noaa.gov/data/gridded/data.UDel_AirT_Prec...

In [43]:
print(sst.values)

<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (time: 1997, lat: 36, lon: 72)
Coordinates:
  * lat      (lat) float32 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
  * lon      (lon) float32 2.5 7.5 12.5 17.5 22.5 ... 342.5 347.5 352.5 357.5
  * time     (time) datetime64[ns] 1856-01-01 1856-02-01 ... 2022-05-01
Data variables:
    sst      (time, lat, lon) object 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    title:          Kaplan Extended SST V2
    platform:       Analysis
    Conventions:    COARDS
    history:        Originally converted from the IRI format to netCDF3 10/20...
    dataset_title:  Kaplan Extended V2 SST anomaly data
    description:    Data is processed at Lamont-Doherty using an \noptimal an...
    References:     https://www.psl.noaa.gov/data/gridded/data.kaplan_sst.html>


In [46]:
print(precip.values)

<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 1332)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 1900-01-01 1900-02-01 ... 2010-12-01
Data variables:
    precip   (time, lat, lon) float32 ...
Attributes:
    history:        created 8/2012 by CAS NOAA/ESRL PSD
    title:          Univ. Delaware Monthly Precipitation, 1900-2010 V3.01
    Conventions:    CF-1.0
    Source:         http://climate.geog.udel.edu/~climate/html_pages/Global2_...
    version:        3.01
    dataset_title:  Terrestrial Air Temperature and Precipitation: 1900-2014 ...
    source:         http://climate.geog.udel.edu/~climate/
    References:     https://www.psl.noaa.gov/data/gridded/data.UDel_AirT_Prec...>


In [ ]:
#### Deixando os dois no mesmo período de tempo

In [48]:
sst = sst.sel(time=slice('1900-01-01', '2010-12-01'))
sst.values

<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (time: 1332, lat: 36, lon: 72)
Coordinates:
  * lat      (lat) float32 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
  * lon      (lon) float32 2.5 7.5 12.5 17.5 22.5 ... 342.5 347.5 352.5 357.5
  * time     (time) datetime64[ns] 1900-01-01 1900-02-01 ... 2010-12-01
Data variables:
    sst      (time, lat, lon) object 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
Attributes:
    title:          Kaplan Extended SST V2
    platform:       Analysis
    Conventions:    COARDS
    history:        Originally converted from the IRI format to netCDF3 10/20...
    dataset_title:  Kaplan Extended V2 SST anomaly data
    description:    Data is processed at Lamont-Doherty using an \noptimal an...
    References:     https://www.psl.noaa.gov/data/gridded/data.kaplan_sst.html>